# Продвинутое машинное обучение: Домашнее задание 2

Второе домашнее задание — самое большое в курсе, в нём придётся и концептуально подумать о происходящем, и технические трудности тоже порешать. Это полноценный проект по анализу данных, начиная от анализа постановки задачи и заканчивая сравнением результатов разных моделей. Задача реальная и серьёзная, хотя тему я выбрал развлекательную: мы будем строить <b>вероятностную рейтинг-систему для спортивного “Что? Где? Когда?” (ЧГК).</b>

<b>Background:</b> в спортивном “Что? Где? Когда?” соревнующиеся команды отвечают на одни и те же вопросы. После минуты обсуждения команды записывают и сдают свои ответы на карточках; побеждает тот, кто ответил на большее число вопросов. Турнир обычно состоит из нескольких десятков вопросов (обычно 36 или 45, иногда 60, больше редко). Часто бывают синхронные турниры, когда на одни и те же вопросы отвечают команды на сотнях игровых площадок по всему миру, т.е. в одном турнире могут играть сотни, а то и тысячи команд. Соответственно, нам нужно:
- построить рейтинг-лист, который способен нетривиально предсказывать результаты будущих турниров;
- при этом, поскольку ЧГК — это хобби, и контрактов тут никаких нет, игроки постоянно переходят из команды в команду, сильный игрок может на один турнир сесть поиграть за другую команду и т.д.; поэтому единицей рейтинг-листа должна быть не команда, а отдельный игрок;
- а что сильно упрощает задачу и переводит её в область домашних заданий на EM-алгоритм — это характер данных: начиная с какого-то момента, в базу результатов начали вносить все повопросные результаты команд, т.е. в данных будут записи вида “какая команда на какой вопрос правильно ответила”.

## 0. Imports

In [77]:
import pickle
import pandas as pd
import numpy as np
import logging
from typing import Any, Dict, Optional, List
from collections import defaultdict
from tqdm import tqdm

from scipy.stats import kendalltau, spearmanr

logger = logging.getLogger()
logger.setLevel(logging.INFO)

## 1. EDA
Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о
составах команд и повопросных результатах (поле mask в results.pkl). Для унификации
предлаестся:
- взять в тренировочный набор турниры с dateStart из 2019 года;
- в тестовый — турниры с dateStart из 2020 года.

In [2]:
!ls data

filtered_results.csv  players.pkl  results.pkl	tournaments.pkl


In [3]:
PLAYERS_PATH = "data/players.pkl"
RESULTS_PATH = "data/results.pkl"
TOURNAMENTS_PATH = "data/tournaments.pkl"

DATE_START = "2019-01-01"
DATE_END = "2021-01-01"

MAX_TEAM_SIZE = 6


def pickle_load(data_path: str) -> Optional[Any]:
    with open(data_path, "rb") as fin:
        data = pickle.load(fin)
    return data


def load_players(data_path: str = PLAYERS_PATH) -> pd.DataFrame:    
    logging.info('Loading players data ...')
    data = pickle_load(data_path)
    df = pd.DataFrame.from_dict(data, orient="index").set_index('id')
    logging.info('Done')
    
    return df


def load_tournaments(data_path: str = TOURNAMENTS_PATH) -> pd.DataFrame:
    logging.info('Loading tournaments data ...')
    data = pickle_load(data_path)
    df = pd.DataFrame.from_dict(data, orient="index").set_index('id')
    
    df[["type_id", "type_name"]] = df["type"].apply(lambda x: list(x.values())).tolist()
    df["season"] = df["season"].apply(lambda x: x.split("/")[-1] if x else None)
    
    drop_columns = ["type", "questionQty", "orgcommittee", "synchData"]
    df = df.drop(columns=drop_columns)
    logging.info('Done')

    return df


def preprocess_results(results: Dict, start_id: int = 0) -> pd.DataFrame:
    data = []
    
    for tournament_id, tournament_data in tqdm(results.items()):
        questions_count = 0
        
        for team_data in tournament_data:
            if team_data.get("mask"):
                questions_count = len(team_data["mask"])
                question_ids = list(range(start_id, start_id + questions_count))
                
                data.append({
                    "tournament_id": tournament_id,
                    "team_id": team_data["team"]["id"],
                    "answered": [int(x) for x in team_data["mask"].replace("X", "0").replace("?", "0")],
                    "question_id": question_ids,
                    "player_id": [player["player"]["id"] for player in team_data["teamMembers"]],
                })
        
        start_id += questions_count
    
    df = pd.DataFrame.from_records(data)

    df = df.explode("player_id", ignore_index=True).apply(
        lambda x: x.explode() if x.name in ("answered", "question_id") else x
    )

    return df


def load_results(data_path: str = RESULTS_PATH, train_ids: List[int] = [], test_ids: List[int] = []) -> Dict:
    logging.info('Loading results data ...')

    data = pickle_load(data_path)
    
    train = {key: data[key] for key in train_ids}
    test = {key: data[key] for key in test_ids}
    
    del data
    
    train_df = preprocess_results(train, start_id = 0)
    test_df = preprocess_results(test, start_id = train_df["question_id"].max() + 1)
    
    logging.info('Done')
    
    return train_df, test_df

In [4]:
players = load_players()
tournaments = load_tournaments()

INFO:root:Loading players data ...
INFO:root:Done
INFO:root:Loading tournaments data ...
INFO:root:Done


In [5]:
train_ids = tournaments[tournaments.dateStart.between("2019-01-01", "2020-01-01")].index.tolist()
test_ids = tournaments[tournaments.dateStart.between("2020-01-01", "2021-01-01")].index.tolist()

train, test = load_results(train_ids=train_ids, test_ids=test_ids)

INFO:root:Loading results data ...
100%|██████████| 418/418 [00:00<00:00, 2162.31it/s]
INFO:root:Done


In [7]:
# Выкинем вопросы, на которые не ответила ни одна команда, либо ответили все

no_answers_questions = np.where(train.groupby("question_id").apply(lambda item: not any(item["answered"])))[0]
all_answers_questions = np.where(train.groupby("question_id").apply(lambda item: all(item["answered"])))[0]

In [8]:
no_answers_questions.shape[0], all_answers_questions.shape[0]

(1786, 741)

In [9]:
train = train[
    ~train["question_id"].isin(no_answers_questions) &
    ~train["question_id"].isin(all_answers_questions)
]

test = test[
    ~test["question_id"].isin(no_answers_questions) &
    ~test["question_id"].isin(all_answers_questions)
]

In [10]:
train.shape, test.shape

((20649927, 5), (4484167, 5))

In [11]:
print(f"Количество турниров: {train.tournament_id.nunique()}")
print(f"Количество игоков: {train.player_id.nunique()}")
print(f"Количество вопросов: {train.question_id.nunique()}")

Количество турниров: 672
Количество игоков: 59097
Количество вопросов: 29576


## 2. Baseline
Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:
- повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
- в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
- для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.

Сделаем ряд допущений, которые позволят нам упростить вычисления:
- Заменим пропуски в ответах ("?" или "X") на "0"
- Не будем использовать вопросы, на которые правильно ответили все команды в турнире, либо ни одна команда не ответила правильно, так как эта информация не поможет нам сравнить команды
- Пусть для игрока с сильой $s_i$ (skill) и сложности вопроса $q_j$ вероятность правильно ответить на этот вопрос равна $p_{ij}$, которая не зависит от силы сокомандников этого игрока, тогда будем обучать логистическую регрессию:

$$\sigma(s_i + q_j) = p_{ij}$$

In [12]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
x_trian = encoder.fit_transform(train[["player_id", "question_id"]])
y_train = train[["answered"]].values.astype(int)

In [13]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(fit_intercept=False, random_state=42, max_iter=500)
clf.fit(x_trian, y_train)

/home/nikolay/anaconda3/envs/ml-mipt/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LogisticRegression(fit_intercept=False, max_iter=500, random_state=42)

In [14]:
ratings = pd.Series({
    player_id: clf.coef_[0][i] for i, player_id in enumerate(encoder.categories_[0])
})

In [15]:
player_id = players[
    (players["name"] == "Юлия") &
    (players["patronymic"] == "Валерьевна") &
    (players["surname"] == "Лазарева")
].index[0]

In [16]:
# Доля игроков, рейтинг которых больше чем у Лазаревой Юлии Валерьевной
(ratings >= ratings[player_id]).mean()

0.009611154353785237

In [17]:
# Игрок с максимильным рейтингом
players.loc[ratings.idxmax()]

name              Максим
patronymic    Михайлович
surname            Руссо
Name: 27403, dtype: object

## 3. Predict

Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому
- предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
- в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров 1 .

Пусть команда дала неправильный ответ в случае, когда ни один из ее участников не дал правильный ответ:

$$P(S) = 1 - \prod_{s \in S} (1 - P(s)) $$

где s - участники команы S. Также для упрощение вычислений возьмем для каждой команды ее лучших 6 игроков

In [62]:
teams = train.groupby("team_id").apply(lambda item: item["player_id"].unique())

In [65]:
MAX_PLAYERS = 6

team_ratings = dict()

for team_id in teams.index.values:
    players_ratings = [ratings[player_id] for player_id in teams.loc[team_id]]
    top_players = players_ratings[:min(MAX_PLAYERS, len(players_ratings))]

    proba = 1 - np.prod([1 - player for player in top_players])
    team_ratings[team_id] = proba

In [67]:
teams_ratings = pd.Series(team_ratings)

In [78]:
test

,tournament_id,team_id,answered,question_id,player_id
0,4957,49804,1,32103,30152
0,4957,49804,1,32104,30152
0,4957,49804,1,32105,30152
0,4957,49804,1,32106,30152
0,4957,49804,1,32107,30152
...,...,...,...,...,...
112869,6456,63129,0,39730,224329
112869,6456,63129,0,39731,224329
112869,6456,63129,0,39732,224329
112869,6456,63129,0,39733,224329


In [71]:
teams_ratings.sort_values()

27001   -7440.008185
75377   -6926.691450
75697   -6925.100885
75685   -6925.100885
74126   -6776.303078
            ...     
64781      14.686905
53702      15.209546
70939      19.220228
71968      38.214198
55837      39.584108
Length: 11739, dtype: float64

In [ ]:
# TODO: add more methods and metrics

## 4. Aggregate

Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
- предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
- разработайте EM-схему для обучения этой модели, реализуйте её в коде;
- обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.

In [ ]:
# TODO

## 5. Rating

А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)? Если будет интересно: постройте топ сложных и простых вопросов со ссылками на конкретные записи в базе вопросов ЧГК https://db.chgk.info/ (это чисто техническое дело, тут никакого ML нету).

In [ ]:
# TODO